### Set the corresponding values in the cell below. Afterwards, just run the following cells

In [7]:
import pathlib
# Class label csv path
labels_csv_path = 'datasets/kinetics_400_labels.csv'
# Datset folder
dataset_root_path = pathlib.Path('datasets')

### Import libraries 

In [8]:
from msclap import CLAP
import torch.nn.functional as F
import numpy as np
import torch
import os
from moviepy.editor import VideoFileClip

### Get Audio from video

In [15]:
video = VideoFileClip(video_path)
if video.audio is None:
    print(f"This video {video_path} has no audio!")
else:
    audio_path = 'datasets/audios' + video_path[8:-3] + 'wav'
    audio_dir = "/".join(audio_path.split("/")[:-1])
    print(audio_dir)
    if not os.path.exists(audio_dir):
            os.makedirs(audio_dir)
    audio_arr = video.audio.write_audiofile(audio_path, verbose=False)
    print(f"Audio saved to {audio_path}")

datasets/audios/Kinetics400/test/crying
MoviePy - Writing audio in datasets/audios/Kinetics400/test/crying/_AUtYTaYMrE_000000_000010.wav


MoviePy - Done.
Audio saved to datasets/audios/Kinetics400/test/crying/_AUtYTaYMrE_000000_000010.wav


### Read class Labels

In [9]:
import csv

label2id = {}
id2label = {}
with open(labels_csv_path, mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)

    for row in csv_reader:
        class_id, class_name = row
        label2id[class_name] = class_id
        id2label[class_id] = class_name

class_labels = list(label2id.keys())

print(f"{len(class_labels)} Unique classes: {class_labels}.")

400 Unique classes: ['abseiling', 'air drumming', 'answering questions', 'applauding', 'applying cream', 'archery', 'arm wrestling', 'arranging flowers', 'assembling computer', 'auctioning', 'baby waking up', 'baking cookies', 'balloon blowing', 'bandaging', 'barbequing', 'bartending', 'beatboxing', 'bee keeping', 'belly dancing', 'bench pressing', 'bending back', 'bending metal', 'biking through snow', 'blasting sand', 'blowing glass', 'blowing leaves', 'blowing nose', 'blowing out candles', 'bobsledding', 'bookbinding', 'bouncing on trampoline', 'bowling', 'braiding hair', 'breading or breadcrumbing', 'breakdancing', 'brush painting', 'brushing hair', 'brushing teeth', 'building cabinet', 'building shed', 'bungee jumping', 'busking', 'canoeing or kayaking', 'capoeira', 'carrying baby', 'cartwheeling', 'carving pumpkin', 'catching fish', 'catching or throwing baseball', 'catching or throwing frisbee', 'catching or throwing softball', 'celebrating', 'changing oil', 'changing wheel', 'c

### CLAP inference with given audio

In [18]:
from tqdm import tqdm

train_paths = list(dataset_root_path.glob("train/*/*.mp4"))

# Load model (Choose between versions '2022' or '2023')
# The model weight will be downloaded automatically if `model_fp` is not specified
clap_model = CLAP(version = '2023', use_cuda=True)

correct = 0
total = 0
for video_path in tqdm(train_paths):
    video_path = video_path.as_posix()
    video = VideoFileClip(video_path)
    if video.audio is None:
        print(f"This video {video_path} has no audio!")
    else:
        audio_path = 'datasets/audios' + video_path[8:-3] + 'wav'
        audio_dir = "/".join(audio_path.split("/")[:-1])
        print(audio_dir)
        if not os.path.exists(audio_dir):
                os.makedirs(audio_dir)
        audio_arr = video.audio.write_audiofile(audio_path, verbose=False)
        print(f"Audio saved to {audio_path}")

    label = video_path.split('/')[-2]

    with torch.no_grad():

        # Extract text embeddings
        text_embeddings = clap_model.get_text_embeddings([f"This is a sound of {c}"for c in class_labels])

        # Extract audio embeddings
        audio_embeddings = clap_model.get_audio_embeddings([audio_path])

        # Compute similarity between audio and text embeddings 
        similarities = clap_model.compute_similarity(audio_embeddings, text_embeddings)

    similarity = F.softmax(similarities, dim=1)
    values, indices = similarity[0].topk(5)
    pred_index = indices[0]

    # Print the results
    print("Ground Truth: {}".format(video_path.split('/')[-2]))
    print("Top predictions:\n")
    for value, index in zip(values, indices):
        print(f"{class_labels[index]:>16s}: {100 * value.item():.2f}%")

    if label == class_labels[pred_index]: 
        correct += 1
    total += 1

print(correct, '/', total)

  0%|          | 0/50 [00:00<?, ?it/s]

datasets/audios/train/crying
MoviePy - Writing audio in datasets/audios/train/crying/9q8mf6GUiiE_000009_000019.wav


  0%|          | 0/50 [00:00<?, ?it/s]

MoviePy - Done.
Audio saved to datasets/audios/train/crying/9q8mf6GUiiE_000009_000019.wav


  2%|▏         | 1/50 [00:04<03:43,  4.56s/it]

...

Ground Truth: crying
Top predictions:

          crying: 67.28%
   carrying baby: 14.02%
   crawling baby: 5.54%
  baby waking up: 5.13%
getting a haircut: 3.95%
datasets/audios/train/crying
MoviePy - Writing audio in datasets/audios/train/crying/_wRQiJdk2Rw_000004_000014.wav


 98%|█████████▊| 49/50 [02:55<00:02,  2.60s/it]

MoviePy - Done.
Audio saved to datasets/audios/train/crying/_wRQiJdk2Rw_000004_000014.wav


100%|██████████| 50/50 [02:57<00:00,  3.55s/it]

Ground Truth: crying
Top predictions:

getting a haircut: 27.44%
dancing macarena: 19.39%
    making sushi: 10.16%
     fixing hair: 6.28%
riding mechanical bull: 3.81%
31 / 50
